In [1]:
import numpy as np
import pandas as pd

In [41]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [3]:
np.random.seed(7)

In [4]:
folder_dir = '../data/'
data_path = "data_clean.csv"
data = pd.read_csv(filepath_or_buffer= folder_dir + data_path, header=0)
data=data.drop('Unnamed: 0',axis=1)

In [5]:
data.shape

(3700449, 37)

In [6]:
data.columns

Index(['date', 'umpcall', 'outcome', 'start_speed', 'pfx_x', 'pfx_z', 'px',
       'pz', 'break_y', 'break_angle', 'break_length', 'pitch_type',
       'spin_dir', 'nasty', 'pitch_count', 'descr', 'y', 'year', 'count_b_p',
       'strikes', 'balls', 'pitcher', 'pitch_type.1', 'rolling_start_speed',
       'long_rolling_start_speed', 'zone_1', 'zone_2', 'zone_3', 'zone_4',
       'zone_5', 'zone_6', 'zone_7', 'zone_8', 'zone_9', 'zone_11', 'zone_12',
       'zone_13'],
      dtype='object')

In [7]:
data['dist']=np.sqrt(data['px']**2+data['pz']**2)
data['speed_diff']=data['rolling_start_speed']-data['long_rolling_start_speed']

In [8]:
pitch_types=pd.get_dummies(data['pitch_type'],prefix='pitch_type')
data=pd.concat([data,pitch_types],axis=1)
pitch_types=0

In [11]:
# model_feats=['px','pz','pfx_x','pfx_z','start_speed',
#              'spin_dir','pitch_count','balls','strikes','date',
#              'y','rolling_start_speed']

model_feats=['px','pz','pfx_x','pfx_z','start_speed','spin_dir',
             'pitch_count','balls','strikes','date',
             'y','speed_diff','dist','pitch_type_CH', 
             'pitch_type_CU','pitch_type_FC', 'pitch_type_FF', 
             'pitch_type_FS', 'pitch_type_FT', 'pitch_type_KC', 
             'pitch_type_SI', 'pitch_type_SL']

# model_feats=['px','pz','pfx_x','pfx_z','y','strikes','date']

In [12]:
import datetime

data["date"] = pd.to_datetime(data["date"])

start = data["date"].searchsorted(datetime.datetime(2012, 1, 1))[0]
#start
end = data["date"].searchsorted(datetime.datetime(2016, 1, 1))[0] - 1
#end

X_train = data[model_feats].loc[start:end]
y_train=X_train['y']
X_train=X_train.drop(['y','date'],axis=1)

X_test=data[model_feats][end:]
y_test=X_test['y']
X_test=X_test.drop(['y','date'],axis=1)
data=0

# Fully Connected Neural Net

In [19]:
X_train.shape

(3221706, 20)

In [20]:
X_train.shape[1]

20

In [14]:
len(model_feats)

22

In [30]:
input_dim = X_train.shape[1]

In [54]:
model = Sequential()
model.add(Dense(30, input_dim=input_dim, activation='relu'))
#model.add(Dropout(rate = 0.1, noise_shape=None, seed=None))
model.add(Dense(20, activation='relu'))
#model.add(Dropout(rate = 0.1, noise_shape=None, seed=None))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [55]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [56]:
model.fit(x = X_train, 
          y = y_train, 
          epochs=50, 
          batch_size=50)

Epoch 1/50
3221706/3221706 [==============================] - 92s 29us/step - loss: 0.5131 - acc: 0.7522
Epoch 2/50
3221706/3221706 [==============================] - 85s 26us/step - loss: 0.4795 - acc: 0.7807
Epoch 3/50
3221706/3221706 [==============================] - 87s 27us/step - loss: 0.4767 - acc: 0.7823
Epoch 4/50
3221706/3221706 [==============================] - 80s 25us/step - loss: 0.4753 - acc: 0.7831
Epoch 5/50
3221706/3221706 [==============================] - 83s 26us/step - loss: 0.4745 - acc: 0.7834
Epoch 6/50
3221706/3221706 [==============================] - 83s 26us/step - loss: 0.4740 - acc: 0.7837
Epoch 7/50
3221706/3221706 [==============================] - 85s 26us/step - loss: 0.4735 - acc: 0.7841
Epoch 8/50
3221706/3221706 [==============================] - 91s 28us/step - loss: 0.4732 - acc: 0.7844
Epoch 9/50
3221706/3221706 [==============================] - 91s 28us/step - loss: 0.4730 - acc: 0.7843
Epoch 10/50
3221706/3221706 [==========================

In [57]:
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

478744/478744 [==============================] - 10s 20us/step

acc: 78.84%


# Neural Net 2

In [54]:
model2 = Sequential()
model2.add(Dense(30, input_dim=input_dim, activation='relu'))
#model.add(Dropout(rate = 0.1, noise_shape=None, seed=None))
model2.add(Dense(30, activation='relu'))
#model.add(Dropout(rate = 0.1, noise_shape=None, seed=None))
model2.add(Dense(20, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

In [55]:
model2.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [56]:
model2.fit(x = X_train, 
          y = y_train, 
          epochs=50, 
          batch_size=50)

Epoch 1/50
3221706/3221706 [==============================] - 92s 29us/step - loss: 0.5131 - acc: 0.7522
Epoch 2/50
3221706/3221706 [==============================] - 85s 26us/step - loss: 0.4795 - acc: 0.7807
Epoch 3/50
3221706/3221706 [==============================] - 87s 27us/step - loss: 0.4767 - acc: 0.7823
Epoch 4/50
3221706/3221706 [==============================] - 80s 25us/step - loss: 0.4753 - acc: 0.7831
Epoch 5/50
3221706/3221706 [==============================] - 83s 26us/step - loss: 0.4745 - acc: 0.7834
Epoch 6/50
3221706/3221706 [==============================] - 83s 26us/step - loss: 0.4740 - acc: 0.7837
Epoch 7/50
3221706/3221706 [==============================] - 85s 26us/step - loss: 0.4735 - acc: 0.7841
Epoch 8/50
3221706/3221706 [==============================] - 91s 28us/step - loss: 0.4732 - acc: 0.7844
Epoch 9/50
3221706/3221706 [==============================] - 91s 28us/step - loss: 0.4730 - acc: 0.7843
Epoch 10/50
3221706/3221706 [==========================

In [ ]:
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))